### Part 1: Saving the visualisation images

In [ ]:
!pip install transformers==4.49.0
!pip install ultralytics
!pip install numpy
!pip install opencv-python
!pip install shap
!pip install torch torchvision torchaudio 
!pip install flash-attn --no-build-isolation
!pip install bitsandbytes accelerate


In [ ]:
import os
import cv2
import numpy as np
import torch
import shap
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import logging
from ultralytics import YOLO
from skimage import color, segmentation
from matplotlib.colors import Normalize
from matplotlib import cm
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq

# Configure logging to log errors
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# Toggle for drawing bounding boxes on final outputs
ADD_BBOX = True

# Initialize your custom-trained YOLOv8 model
model = YOLO("")

# Class index to class name mapping
class_name_mapping = {0: "part_1", 1: "part_2", 2: "part_3"}

###############################################################################
# 1. Modified DRISE Pipeline with Bounding Box Toggle
###############################################################################
def generate_drise_saliency_per_class(model, image_path, output_directory, label_names,
                                        n_masks=10000, grid_size=(16, 16), prob_thresh=0.2,
                                        saliency_threshold=0, draw_bbox=False, batch_size=32):
    os.makedirs(output_directory, exist_ok=True)
    image = cv2.imread(image_path)
    if image is None:
        logging.error(f"Failed to read image: {image_path}")
        return
    image_h, image_w = image.shape[:2]

    # Run YOLO on the image and extract predictions
    results = model(image)
    preds = results[0].boxes.xyxy.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    pred_classes = results[0].boxes.cls.cpu().numpy().astype(int)

    # Get the highest-confidence box per class
    best_boxes = {}
    for i in range(len(preds)):
        class_id = pred_classes[i]
        confidence = scores[i]
        bbox = preds[i]
        if class_id not in best_boxes or confidence > best_boxes[class_id]['confidence']:
            best_boxes[class_id] = {'bbox': bbox, 'confidence': confidence}

    # For each detected class, generate a saliency map
    for class_id, info in best_boxes.items():
        if class_id >= len(label_names):
            logging.warning(f"Class ID {class_id} exceeds label names length. Skipping.")
            continue
        target_box = info['bbox']
        class_name = label_names[class_id]

        saliency_map = np.zeros((image_h, image_w), dtype=np.float32)
        batch_masks = []  # accumulate masks for batched inference

        for i in range(n_masks):
            mask = generate_mask((image_w, image_h), grid_size, prob_thresh)
            batch_masks.append(mask)
            # When we have a full batch or are at the last mask, run batched inference:
            if len(batch_masks) == batch_size or i == n_masks - 1:
                masked_images = [mask_image(image, m) for m in batch_masks]
                batch_results = model.predict(source=masked_images, save=False, verbose=False)
                for m, result in zip(batch_masks, batch_results):
                    if len(result.boxes) > 0:
                        masked_preds = result.boxes.xyxy.cpu().numpy()
                        masked_scores = result.boxes.conf.cpu().numpy()
                        masked_classes = result.boxes.cls.cpu().numpy().astype(int)
                        for box, score, cls in zip(masked_preds, masked_scores, masked_classes):
                            if cls == class_id:
                                iou_score = iou(target_box, box)
                                saliency_map += m * iou_score * score
                batch_masks = []  # clear batch

        # Normalize and threshold the saliency map
        saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min() + 1e-6)
        saliency_map[saliency_map < saliency_threshold] = 0

        heatmap = cv2.applyColorMap((saliency_map * 255).astype(np.uint8), cv2.COLORMAP_JET)
        cam = cv2.addWeighted(image, 0.5, heatmap, 0.5, 0)

        # Draw bounding box if enabled (only the box, no text)
        if draw_bbox:
            x_min, y_min, x_max, y_max = map(int, target_box)
            cv2.rectangle(cam, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        image_name = os.path.splitext(os.path.basename(image_path))[0]
        output_path = os.path.join(output_directory, f"{image_name}_{class_name}_saliency.png")
        cv2.imwrite(output_path, cam)


# Supporting functions for DRISE
def generate_mask(image_size, grid_size, prob_thresh):
    image_w, image_h = image_size
    grid_w, grid_h = grid_size
    cell_w, cell_h = math.ceil(image_w / grid_w), math.ceil(image_h / grid_h)
    up_w, up_h = (grid_w + 1) * cell_w, (grid_h + 1) * cell_h
    mask = (np.random.uniform(0, 1, size=(grid_h, grid_w)) < prob_thresh).astype(np.float32)
    mask = cv2.resize(mask, (up_w, up_h), interpolation=cv2.INTER_LINEAR)
    offset_w = np.random.randint(0, cell_w)
    offset_h = np.random.randint(0, cell_h)
    return mask[offset_h:offset_h + image_h, offset_w:offset_w + image_w]

def mask_image(image, mask):
    return ((image.astype(np.float32) / 255 * np.dstack([mask] * 3)) * 255).astype(np.uint8)

def iou(box1, box2):
    box1 = np.array(box1)
    box2 = np.array(box2)
    tl = np.maximum(box1[:2], box2[:2])
    br = np.minimum(box1[2:], box2[2:])
    intersection = np.prod(np.maximum(0, br - tl))
    area1 = np.prod(box1[2:] - box1[:2])
    area2 = np.prod(box2[2:] - box2[:2])
    return intersection / (area1 + area2 - intersection + 1e-6)

###############################################################################
# 2. Modified SHAP Pipeline (Superpixel-based) with Bounding Box Toggle
###############################################################################
def run_shap_on_slic_for_classes(image_path, output_dir,
                                 k=20, m=10, num_iter=5,
                                 final_size=(256, 256), nsamples=50,
                                 add_boundaries=False, draw_bbox=False):
    ensure_dir(output_dir)

    original_bgr = cv2.imread(image_path)
    if original_bgr is None:
        raise ValueError(f"Could not read image: {image_path}")
    img_name = os.path.splitext(os.path.basename(image_path))[0]

    # Run YOLO on the full-size image and save a check image with boxes
    results = model.predict(source=[original_bgr], save=False, verbose=False, imgsz=512)
    result = results[0]
    boxed_image = draw_yolo_boxes(original_bgr, result, model)
    bbox_save_path = os.path.join(output_dir, f"{img_name}_yolo_bboxes.jpg")
    cv2.imwrite(bbox_save_path, boxed_image)
    print(f"[INFO] Saved YOLO bbox image => {bbox_save_path}")

    if len(result.boxes) == 0:
        print("[INFO] No objects detected in the image.")
        return

    # Determine the top 3 classes by highest confidence
    cls_array = result.boxes.data[:, 5].cpu().numpy()
    conf_array = result.boxes.data[:, 4].cpu().numpy()
    unique_classes = np.unique(cls_array)
    class_conf_map = {int(uc): conf_array[cls_array == uc].max() for uc in unique_classes}
    sorted_by_conf = sorted(class_conf_map.items(), key=lambda x: x[1], reverse=True)
    top3_classes = [cls for cls, _ in sorted_by_conf[:3]]
    print(f"[INFO] Top 3 classes by confidence => {top3_classes}")

    # Resize image for SLIC and SHAP explanation
    resized_bgr = cv2.resize(original_bgr, final_size)
    resized_rgb = cv2.cvtColor(resized_bgr, cv2.COLOR_BGR2RGB)
    img_lab = color.rgb2lab(resized_rgb)

    # Run SLIC segmentation to obtain superpixels
    clusters, label_map = slic_segmentation(img_lab, k=k, m=m, num_iter=num_iter)
    num_superpixels = len(clusters)
    print(f"[INFO] Found {num_superpixels} superpixels with k={k}, m={m}, num_iter={num_iter}.")

    # For each of the top 3 classes, run SHAP explanation
    for target_class in top3_classes:
        class_name = model.names[target_class]
        print(f"[SHAP] Generating explanation for class: {class_name} (ID={target_class})")

        background = np.zeros((1, num_superpixels))
        explainer = shap.KernelExplainer(
            lambda mask_matrix: superpixel_shap_predict_for_class(mask_matrix, resized_bgr, label_map, target_class),
            background
        )
        test_sample = np.ones((1, num_superpixels))
        shap_values = explainer.shap_values(test_sample, nsamples=nsamples)
        sv = shap_values[0][0] if isinstance(shap_values, list) else shap_values[0]

        overlay = np.zeros(label_map.shape, dtype=float)
        for sp_index in range(num_superpixels):
            overlay[label_map == sp_index] = sv[sp_index]

        # Create a grayscale background for blending
        resized_gray = cv2.cvtColor(resized_rgb, cv2.COLOR_RGB2GRAY)
        resized_gray_rgb = cv2.cvtColor(resized_gray, cv2.COLOR_GRAY2RGB)

        cmap = cm.colormaps['bwr'] if hasattr(cm, 'colormaps') else cm.get_cmap('bwr')
        norm = Normalize(vmin=overlay.min(), vmax=overlay.max())
        overlay_rgba = cmap(norm(overlay))
        threshold = 0.1 * (overlay.max() - overlay.min())
        low_contribution_mask = np.abs(overlay) < threshold
        overlay_rgb = (overlay_rgba[..., :3] * 255).astype(np.uint8)
        alpha = 0.5

        blended = resized_gray_rgb.copy()
        contributing_mask = ~low_contribution_mask
        for c in range(3):
            blended[..., c] = np.where(
                contributing_mask,
                (1.0 - alpha) * resized_gray_rgb[..., c] + alpha * overlay_rgb[..., c],
                resized_gray_rgb[..., c],
            )

        if add_boundaries:
            boundaries = segmentation.find_boundaries(label_map, mode='outer')
            blended[boundaries] = [0, 0, 0]

        # If toggled, draw the bounding box for this target class on the blended image.
        if draw_bbox:
            best_box = None
            best_conf = -1
            for b in result.boxes.data:
                b_cpu = b.cpu().numpy()
                x1, y1, x2, y2, conf, cls_id = b_cpu
                if int(cls_id) == target_class and conf > best_conf:
                    best_conf = conf
                    best_box = (x1, y1, x2, y2)
            if best_box is not None:
                scale_x = final_size[0] / original_bgr.shape[1]
                scale_y = final_size[1] / original_bgr.shape[0]
                scaled_box = (int(best_box[0] * scale_x), int(best_box[1] * scale_y),
                              int(best_box[2] * scale_x), int(best_box[3] * scale_y))
                cv2.rectangle(blended, (scaled_box[0], scaled_box[1]), (scaled_box[2], scaled_box[3]),
                              (0, 255, 0), 2)
                cv2.putText(blended, f"{class_name} {best_conf:.2f}",
                            (scaled_box[0], scaled_box[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        output_path = os.path.join(output_dir, f"{img_name}_{k}_{num_iter}_{nsamples}_{class_name}_shap.jpg")
        cv2.imwrite(output_path, cv2.cvtColor(blended, cv2.COLOR_RGB2BGR))
        print(f"[SHAP] Saved superpixel explanation => {output_path}")

        torch.cuda.empty_cache()

# Additional supporting functions for the superpixel SHAP pipeline
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def draw_yolo_boxes(image_bgr, result, model):
    draw_img = image_bgr.copy()
    if len(result.boxes) == 0:
        return draw_img
    boxes_data = result.boxes.data.cpu().numpy()
    classes = boxes_data[:, 5].astype(int)
    confidences = boxes_data[:, 4]
    best_boxes = {}
    for i, cls in enumerate(classes):
        if cls not in best_boxes or confidences[i] > best_boxes[cls][4]:
            best_boxes[cls] = boxes_data[i]
    for cls, box in best_boxes.items():
        x1, y1, x2, y2, conf, cls_id = box
        class_name = model.names[int(cls_id)]
        cv2.rectangle(draw_img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        text = f"{class_name} {conf:.2f}"
        cv2.putText(draw_img, text, (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    return draw_img

def superpixel_shap_predict_for_class(mask_matrix, original_image_bgr, label_map, target_class):
    batch_size = mask_matrix.shape[0]
    images = []
    for i in range(batch_size):
        mask = mask_matrix[i]
        masked_img = mask_superpixels(original_image_bgr, label_map, mask)
        images.append(masked_img)
    confs = yolo_predict_for_class(images, target_class)
    return confs

def yolo_predict_for_class(images_bgr, target_class):
    results = model.predict(source=images_bgr, save=False, verbose=False, imgsz=512)
    confidences = []
    for result in results:
        if len(result.boxes) > 0:
            cls_array = result.boxes.data[:, 5].cpu().numpy()
            conf_array = result.boxes.data[:, 4].cpu().numpy()
            filtered_indices = (cls_array == target_class)
            filtered_boxes = conf_array[filtered_indices]
            conf = float(filtered_boxes.max()) if len(filtered_boxes) > 0 else 0.0
        else:
            conf = 0.0
        confidences.append(conf)
    return np.array(confidences)

def mask_superpixels(image_bgr, label_map, mask):
    masked_img = np.zeros_like(image_bgr, dtype=np.uint8)
    num_superpixels = mask.shape[0]
    for sp_index in range(num_superpixels):
        if mask[sp_index] == 1:
            masked_img[label_map == sp_index] = image_bgr[label_map == sp_index]
    return masked_img

def slic_segmentation(img_lab, k=100, m=20, num_iter=10):
    img_h, img_w = img_lab.shape[:2]
    N = img_h * img_w
    S = int(math.sqrt(N / k))
    clusters = []
    tag = {}
    dis = np.full((img_h, img_w), np.inf)
    clusters = initial_cluster_center(S, img_lab, img_h, img_w, clusters)
    reassign_cluster_center_acc_to_grad(clusters, img_lab, img_h, img_w)
    for _ in range(num_iter):
        assign_pixels_to_cluster(clusters, S, img_lab, img_h, img_w, tag, dis, m)
        update_cluster_mean(clusters, img_lab)
    label_map = np.zeros((img_h, img_w), dtype=np.int32)
    for idx, c in enumerate(clusters):
        for (ph, pw) in c.pixels:
            label_map[ph, pw] = idx
    return clusters, label_map

def initial_cluster_center(S, img_lab, img_h, img_w, clusters):
    h = S // 2
    w_init = S // 2
    while h < img_h:
        w = w_init
        while w < img_w:
            clusters.append(make_superPixel(h, w, img_lab))
            w += S
        h += S
    return clusters

def make_superPixel(h, w, img_lab):
    return SuperPixels(h, w, img_lab[h, w, 0], img_lab[h, w, 1], img_lab[h, w, 2])

class SuperPixels(object):
    def __init__(self, h, w, l=0, a=0, b=0):
        self.update(h, w, l, a, b)
        self.pixels = []
    def update(self, h, w, l, a, b):
        self.h = h
        self.w = w
        self.l = l
        self.a = a
        self.b = b

def calc_gradient(h, w, img_lab, img_w, img_h):
    if w + 1 >= img_w:
        w = img_w - 2
    if h + 1 >= img_h:
        h = img_h - 2
    grad = ((img_lab[h + 1, w + 1, 0] - img_lab[h, w, 0]) +
            (img_lab[h + 1, w + 1, 1] - img_lab[h, w, 1]) +
            (img_lab[h + 1, w + 1, 2] - img_lab[h, w, 2]))
    return abs(grad)

def reassign_cluster_center_acc_to_grad(clusters, img_lab, img_h, img_w):
    for c in clusters:
        base_grad = calc_gradient(c.h, c.w, img_lab, img_w, img_h)
        best_h, best_w = c.h, c.w
        best_grad = base_grad
        for dh in range(-1, 2):
            for dw in range(-1, 2):
                H = c.h + dh
                W = c.w + dw
                if 0 <= H < img_h and 0 <= W < img_w:
                    new_grad = calc_gradient(H, W, img_lab, img_w, img_h)
                    if new_grad < best_grad:
                        best_h, best_w = H, W
                        best_grad = new_grad
        if (best_h, best_w) != (c.h, c.w):
            c.update(best_h, best_w,
                     img_lab[best_h, best_w, 0],
                     img_lab[best_h, best_w, 1],
                     img_lab[best_h, best_w, 2])

def assign_pixels_to_cluster(clusters, S, img_lab, img_h, img_w, tag, dis, m):
    for c in clusters:
        row_start = max(c.h - 2 * S, 0)
        row_end = min(c.h + 2 * S, img_h)
        col_start = max(c.w - 2 * S, 0)
        col_end = min(c.w + 2 * S, img_w)
        for h in range(row_start, row_end):
            for w in range(col_start, col_end):
                L, A, B = img_lab[h, w]
                Dc = math.sqrt((L - c.l) ** 2 + (A - c.a) ** 2 + (B - c.b) ** 2)
                Ds = math.sqrt((h - c.h) ** 2 + (w - c.w) ** 2)
                D = math.sqrt((Dc / m) ** 2 + (Ds / S) ** 2)
                if D < dis[h, w]:
                    if (h, w) in tag:
                        old_cluster = tag[(h, w)]
                        if (h, w) in old_cluster.pixels:
                            old_cluster.pixels.remove((h, w))
                    tag[(h, w)] = c
                    c.pixels.append((h, w))
                    dis[h, w] = D

def update_cluster_mean(clusters, img_lab):
    for c in clusters:
        if len(c.pixels) == 0:
            continue
        sum_h = 0
        sum_w = 0
        sum_L = 0.0
        sum_A = 0.0
        sum_B = 0.0
        for (ph, pw) in c.pixels:
            sum_h += ph
            sum_w += pw
            sum_L += img_lab[ph, pw, 0]
            sum_A += img_lab[ph, pw, 1]
            sum_B += img_lab[ph, pw, 2]
        count = len(c.pixels)
        mean_h = sum_h // count
        mean_w = sum_w // count
        mean_L = sum_L / count
        mean_A = sum_A / count
        mean_B = sum_B / count
        c.update(mean_h, mean_w, mean_L, mean_A, mean_B)

###############################################################################
# 3. Main Loop: Process all images using the modified pipelines
###############################################################################

input_dir = ''
output_dir = ""

os.makedirs(output_dir, exist_ok=True)

# Process images with a tqdm progress bar and error logging
for image_name in tqdm(sorted(os.listdir(input_dir)), desc="Processing images"):
    image_path = os.path.join(input_dir, image_name)
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        try:
            generate_drise_saliency_per_class(model, image_path, output_dir, 
                                              list(class_name_mapping.values()),
                                              n_masks=1000, saliency_threshold=0, 
                                              draw_bbox=ADD_BBOX)

            run_shap_on_slic_for_classes(image_path, output_dir,
                                         final_size=(512, 512), nsamples=100,
                                         k=40, m=10, num_iter=20,
                                         add_boundaries=False, draw_bbox=ADD_BBOX)
        except Exception as e:
            logging.error(f"Error processing {image_path}: {e}")

print("DRISE and SHAP for all images.")


### Part 2: Large Multimodal Model Interaction

In [ ]:
# --------------------------------------------------
# 1. Logging Setup
# --------------------------------------------------
# Suppress INFO-level logs from accelerate and transformers
logging.getLogger("accelerate").setLevel(logging.WARNING)
logging.getLogger("transformers").setLevel(logging.WARNING)
# Optionally, set environment variables for transformers verbosity:
os.environ["TRANSFORMERS_VERBOSITY"] = "error"

# --------------------------------------------------
# 2. Global Config & Model Load
# --------------------------------------------------
DEVICE = "cuda:0"  # or just "cuda" if you have one GPU
model_name = "HuggingFaceM4/Idefics2-8b"  # Make sure you have access to this model

# Standard default image path (adjust as needed)
DEFAULT_IMAGE_PATH = ""

try:
    processor = AutoProcessor.from_pretrained(
        model_name,
        trust_remote_code=True
    )

    model = AutoModelForVision2Seq.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2",  # remove if you get errors
        device_map="auto",
        trust_remote_code=True
    )

except torch.cuda.OutOfMemoryError:
    print("OutOfMemoryError while loading the model. Attempting to free cache...")
    torch.cuda.empty_cache()
    raise

# --------------------------------------------------
# 3. Prompt & Generation Functions
# --------------------------------------------------
def generate_prompt_for_eval(question: str) -> str:
    """
    Builds a system+user prompt with a delimiter ("### Answer:") appended
    so that only the answer portion is extracted from the generation.
    """
    system_prompt = f"""

    Context:
    This image is the Saliency Map Visualization which emphasizes the areas of the image that the model finds most critical for its decision-making process.
    By highlighting the most 'important' pixels, the saliency map allows us to see which regions of the image had the most influence on the model predictions.
    It visually demonstrates how the model identifies and delineates the target object in relation to the surrounding elements.
    Instructions for the Response:
     - Focus on what the saliency map reveals about the object's location or features, without referencing any color details.
     - Provide a clear and simple explanation.
     - Use straightforward language to describe how the visualization contributes to understanding the detection.
     - Limit the output to 10-20 words.
  
    ### Answer:
    """
    user_text = question if question else "Hello!"
    
    prompt = [
        {
            "role": "sytem",
            "content": [{"type": "text", "text": system_prompt.strip()}]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": user_text}]
        }
    ]
    return prompt

def ask_model_eval(question: str, image_path: str = DEFAULT_IMAGE_PATH):
    """
    1) Builds a prompt (using a default image path).
    2) Passes it to the model for generation.
    3) Extracts and returns only the answer after the delimiter.
    """
    try:
        # Build chat-like prompt with delimiter
        messages = generate_prompt_for_eval(question)
        
        # If the default image exists, insert it into the prompt
        if image_path and os.path.exists(image_path):
            messages[1]["content"].insert(0, {"type": "image", "image": Image.open(image_path)})

        # Apply chat template logic to build the prompt text
        prompt_text = processor.apply_chat_template(messages, add_generation_prompt=False)

        # Prepare inputs, including image if available
        if image_path and os.path.exists(image_path):
            inputs = processor(
                text=prompt_text,
                images=[Image.open(image_path)],
                return_tensors="pt"
            ).to(DEVICE)
        else:
            inputs = processor(
                text=prompt_text,
                return_tensors="pt"
            ).to(DEVICE)

        # Generate output
        pixel_values = inputs.get("pixel_values", None)
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=pixel_values,  # None if no image
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7
        )

        if generated_ids is None or len(generated_ids) == 0:
            print("No output generated.")
            return None

        output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Extract only the answer after the delimiter "### Answer:"
        if "### Answer:" in output_text:
            final_reply = output_text.split("### Answer:")[-1].strip()
        else:
            final_reply = output_text.strip()

        if not final_reply:
            return None

        return final_reply

    except torch.cuda.OutOfMemoryError as oom_e:
        print("CUDA OutOfMemoryError encountered during generation!")
        torch.cuda.empty_cache()
        raise oom_e
    except Exception as e:
        print(f"Error in ask_model_eval: {e}")
        return None
    finally:
        torch.cuda.empty_cache()

# --------------------------------------------------
# 4. Widgets for Interactive Demo
# --------------------------------------------------

# (A) Text Input for the question (only question input now)
question_input = widgets.Text(
    value="",
    placeholder="Type your question here",
    description="Question:",
    layout=widgets.Layout(width="90%")
)

# (B) Button to trigger generation
eval_button = widgets.Button(
    description="Get Model Answer",
    button_style="success"
)

# (C) Output area
out = widgets.Output()

# Button callback
def on_button_clicked(_):
    with out:
        clear_output()
        question = question_input.value

        print(f"\n--- Generating answer for question: '{question}' ---")
        response = ask_model_eval(question)  # Uses default image path

        print("\nModel Answer:")
        if response:
            print(response)
        else:
            print("No valid response was generated. Check for OOM or model issues.")

eval_button.on_click(on_button_clicked)

# --------------------------------------------------
# 5. Display the Widget
# --------------------------------------------------
display(question_input, eval_button, out)
